In this notebook we process a large part of the glaciers in the Alps. The image is chosen in such a way that the
most important glaciers in France/Austria/Switzerland/Italy are covered.

In [1]:
# Import script to embed geolocation.
from IPython.display import HTML
import datetime

import sys
sys.path.extend(['/Users/gertmulder/software/rippl_main'])

from rippl.orbit_geometry.read_write_shapes import ReadWriteShapes
from rippl.SAR_sensors.sentinel.sentinel_download import DownloadSentinel
from rippl.processing_templates.general_sentinel_1 import GeneralPipelines

glacier_shape = [[11.260986328125, 46.84516443029276],
            [12.095947265625, 46.803819640791545],
            [13.073730468750002, 46.830133640447386],
            [13.5955810546875, 46.9465122958623],
            [13.403320312499998, 47.204642388766935],
            [10.607299804687523, 47.247542522267985],
            [9.6240234375, 46.927758623434435],
            [8.827514648437523, 47.00835281693702],
            [7.772827148437526, 46.63246472462745],
            [6.701660156249999, 46.118941506107056],
            [6.597290039062523, 45.673563046842496],
            [6.0150146484375, 45.263288531496855],
            [6.0479736328125, 44.762336674810996],
            [6.43798828125, 44.793530904744046],
            [6.6796875, 44.98034238084973],
            [7.2894287109375,45.28261705751738],
            [7.3663330078125, 45.4947963896697],
            [8.052978515625, 45.84028105450088],
            [8.316650390625, 46.20264638061017],
            [8.9208984375, 46.244451065485066],
            [9.563598632812523, 46.1246527020117],
            [10.2557373046875, 46.1912395780416],
            [10.838012695312523, 46.50406399740095],
            [11.260986328125, 46.84516443029276]]
study_area = ReadWriteShapes()
study_area(glacier_shape)

geojson = study_area.shape

# Try to do the same by creating a shapefile with QGIS, geojson online or a .kml file in google earth.
# study_area.read_kml(kml_path)
# study_area.read_geo_json(geojson_path)
# study_area.read_shapefile(shapefile_path)

The next step in the processing is selection of the right track, as the Sentinel-1 satellite will cover the area on both
ascending and descending tracks. However, to do so, we will have to define the start and end date of our data search,
because the satellite is not always acquiring data.
Next search will give a small oversight of the available tracks during our period of interest. In our case we will
search one week before and one week after the earthquake.

Watch out with extending the search window, this can result in a lot of images. Note that every downloaded file will take approximately 4GB of disk space!

In [2]:
# First we search using a time window

# Track and data type of Sentinel data
mode = 'IW'
product_type = 'SLC'
polarisation = 'VV'

# First we check using a time window
start_date = datetime.datetime(year=2018, month=9, day=1)
end_date = datetime.datetime(year=2018, month=10, day=1)

# find_track = DownloadSentinel(start_date=start_date, end_date=end_date, shape=study_area.shape, sensor_mode=mode,
#                              polarisation=polarisation)
# find_track.sentinel_search_ASF()
# find_track.summarize_search_results(plot_cartopy=True, buffer=2)

After selection of the right track we can start the actual download of the images. In our case we use track 66.

This will download our data automatically to our radar database. Additionally, it will download the precise orbit files.
These files are created within a few weeks after the data acquisition and define the satellite orbit within a few cm
accuracy. These orbits are necessary to accurately define the positions of the radar pixels on the ground later on
in the processing.


In [3]:
# Track and data type of Sentinel data
mode = 'IW'
product_type = 'SLC'
polarisation = 'VV'

from rippl.processing_templates.general_sentinel_1 import GeneralPipelines

# Create the list of the 4 different stacks.
track_no = 66
stack_name = 'glaciers_Alps'
# For every track we have to select a master date. This is based on the search results earlier.
# Choose the date with the lowest coverage to create an image with only the overlapping parts.
master_date = datetime.datetime(year=2018, month=8, day=20)

# Stack test
start_date = datetime.datetime(year=2018, month=8, day=19)
end_date = datetime.datetime(year=2018, month=8, day=30)

# Number of processes for parallel processing. Make sure that for every process at least 2GB of RAM is available
no_processes = 4

s1_processing = GeneralPipelines(processes=no_processes)
#s1_processing.download_sentinel_data(start_date=start_date, end_date=end_date, track=track_no,
#                                           polarisation=polarisation, shapefile=study_area.shape, data=True, source='ASF')
#s1_processing.create_sentinel_stack(start_date=start_date, end_date=end_date, master_date=master_date, cores=no_processes,
#                                          track=track_no,stack_name=stack_name, polarisation=polarisation,
#                                          shapefile=study_area.shape, mode=mode, product_type=product_type)

# Finally load the stack itself. If you want to skip the download step later, run this line before other steps!
s1_processing.read_stack(start_date=start_date, end_date=end_date, stack_name=stack_name)

To define the location of the radar pixels on the ground we need the terrain elevation. Although it is possible to 
derive terrain elevation from InSAR data, our used Sentinel-1 dataset is not suitable for this purpose. Therefore, we
download data from an external source to create a digital elevation model (DEM). In our case we use SRTM data. 

However, to find the elevation of the SAR data grid, we have to resample the data to the radar grid first to make it
usable. This is done in the next steps.

In [5]:
# Some basic settings for DEM creation.
dem_buffer = 0.1        # Buffer around radar image where DEM data is downloaded
dem_rounding = 0.1      # Rounding of DEM size in degrees
dem_type = 'SRTM3'      # DEM type of data we download (SRTM1, SRTM3 and TanDEM-X are supported)

# Define both the coordinate system of the full radar image and imported DEM
s1_processing.create_radar_coordinates()
s1_processing.create_dem_coordinates(dem_type=dem_type)

# Download external DEM
s1_processing.download_external_dem(buffer=dem_buffer, rounding=dem_rounding, n_processes=no_processes)

Using the obtained elevation model the exact location of the radar pixels in cartesian (X,Y,Z) and geographic (Lat/Lon)
can be derived. This is only done for the master or reference image. This process is referred to as geocoding.

In [5]:
# Geocoding of image
s1_processing.geocoding()

Processing pipeline block 1 out of 4
Processing pipeline block 2 out of 4
Processing pipeline block 3 out of 4
Processing pipeline block 4 out of 4
Processing pipeline block 1 out of 4
Processing pipeline block 2 out of 4
Processing pipeline block 3 out of 4
Processing pipeline block 4 out of 4
Concatenated dataset already exists. If you want to overwrite set overwrite to True
Concatenated dataset already exists. If you want to overwrite set overwrite to True
Finished concatenation geocode of /Users/gertmulder/SAR_processing/radar_data_stacks/sentinel1/glaciers_Alps/20180820
Finished concatenation geocode of /Users/gertmulder/SAR_processing/radar_data_stacks/sentinel1/glaciers_Alps/20180820
Concatenated dataset already exists. If you want to overwrite set overwrite to True


The information from the geocoding can directly be used to find the location of the master grid pixels in the slave
grid images. This process is called coregistration. Because the orbits are not exactly the same with every satellite 
overpass but differ hundreds to a few thousand meters every overpass, the grids are slightly shifted with respect to 
each other. These shift are referred to as the spatial baseline of the images. To correctly overlay the master and slave
images the software coregisters and resamples to the master grid.

To do so the following steps are done:
1. Coregistration of slave to master image
2. Deramping the doppler effects due to TOPs mode of Sentinel-1 satellite
3. Resampling of slave image
4. Reramping resampled slave image.

Due to the different orbits of the master and slave image, the phase of the radar signal is also shifted. We do not 
know the exact shift of the two image, but using the geometry of the two images we can estimate the shift of the phase
between different pixels. Often this shift is split in two contributions:
1. The flat earth phase. This phase is the shift in the case the earth was a perfect ellipsoid
2. The topographic phase. This is the phase shift due to the topography on the ground.
In our processing these two corrections are done in one go.

In [ ]:
# Next step applies resampling and phase correction in one step.
# Polarisation
s1_processing.geometric_coregistration_resampling(polarisation=polarisation, output_phase_correction=True)

Now we can create calibrated amplitudes, interferograms and coherences.

In [6]:
for dist in [50, 100, 200]:

    # The actual creation of the calibrated amplitude images
    s1_processing.create_ml_coordinates(standard_type='oblique_mercator', dx=dist, dy=dist, buffer=0, rounding=0)
    # s1_processing.prepare_multilooking_grid(polarisation)
    s1_processing.create_calibrated_amplitude_multilooked(polarisation)
    s1_processing.create_output_tiffs_amplitude()

    s1_processing.create_ifg_network(temporal_baseline=15)
    s1_processing.create_interferogram_multilooked(polarisation)
    s1_processing.create_coherence_multilooked(polarisation)

    # Create output geotiffs
    s1_processing.create_output_tiffs_coherence_ifg()

    # Create lat/lon/incidence angle/DEM for multilooked grid.
    s1_processing.create_geometry_mulitlooked(baselines=True)
    s1_processing.create_output_tiffs_geometry()

Processing pipeline block 1 out of 1
Start time block 1 is 2020-10-27 14:34:32.702496
Skipping processing. Process already finished
Skipping processing. Process already finished
Finished processing pipeline block 1 out of 1
Finished block 1 in 0:00:00.145344
File /Users/gertmulder/SAR_processing/radar_data_products/sentinel1/glaciers_Alps/calibrated_amplitude/proj_oblique_mercator_50_50/20180820_calibrated_amplitude_db_VV@proj_oblique_mercator_50_50_in_coor_radar.tiff does already exist
File /Users/gertmulder/SAR_processing/radar_data_products/sentinel1/glaciers_Alps/calibrated_amplitude/proj_oblique_mercator_50_50/20180820_no_of_looks_VV@proj_oblique_mercator_50_50_in_coor_radar.tiff does already exist
File /Users/gertmulder/SAR_processing/radar_data_products/sentinel1/glaciers_Alps/calibrated_amplitude/proj_oblique_mercator_50_50/20180826_calibrated_amplitude_db_VV@proj_oblique_mercator_50_50_in_coor_radar.tiff does already exist
Processing pipeline block 1 out of 1
Start time block 